In [17]:
# MIMIC-IV Demo Dataset Preprocessing
# This notebook provides comprehensive preprocessing for the MIMIC-IV clinical database demo

import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Define paths
DATA_PATH = Path("datasets/mimic-iv-demo")
HOSP_PATH = DATA_PATH / "hosp"
ICU_PATH = DATA_PATH / "icu"
OUTPUT_PATH = Path("output")
OUTPUT_PATH.mkdir(exist_ok=True)

print("=" * 80)
print("MIMIC-IV Demo Dataset Preprocessing Pipeline")
print("=" * 80)
print(f"\nData Path: {DATA_PATH}")
print(f"Output Path: {OUTPUT_PATH}")
print("\nDataset Overview:")
print("- 100 demo patients from MIMIC-IV")
print("- Hospital (hosp) module: Patient demographics, admissions, diagnoses, procedures, prescriptions, labs, etc.")
print("- ICU module: ICU stays, chartevents, inputevents, outputevents, procedures, etc.")


MIMIC-IV Demo Dataset Preprocessing Pipeline

Data Path: datasets\mimic-iv-demo
Output Path: output

Dataset Overview:
- 100 demo patients from MIMIC-IV
- Hospital (hosp) module: Patient demographics, admissions, diagnoses, procedures, prescriptions, labs, etc.
- ICU module: ICU stays, chartevents, inputevents, outputevents, procedures, etc.


In [18]:
# ==============================================================================
# 1. LOAD CORE PATIENT AND ADMISSION DATA
# ==============================================================================

print("\n" + "="*80)
print("STEP 1: Loading Core Patient and Admission Data")
print("="*80)

# Load patient demographics
patients = pd.read_csv(HOSP_PATH / "patients.csv.gz")
print(f"\n✓ Patients loaded: {len(patients):,} records, {patients['subject_id'].nunique()} unique patients")

# Load admissions
admissions = pd.read_csv(HOSP_PATH / "admissions.csv.gz")
print(f"✓ Admissions loaded: {len(admissions):,} records, {admissions['hadm_id'].nunique()} unique admissions")

# Convert datetime columns
datetime_cols_admissions = ['admittime', 'dischtime', 'deathtime', 'edregtime', 'edouttime']
for col in datetime_cols_admissions:
    admissions[col] = pd.to_datetime(admissions[col])

# Calculate length of stay
admissions['los_days'] = (admissions['dischtime'] - admissions['admittime']).dt.total_seconds() / (24 * 3600)

print(f"\nAdmission Statistics:")
print(f"- Average LOS: {admissions['los_days'].mean():.2f} days")
print(f"- Median LOS: {admissions['los_days'].median():.2f} days")
print(f"- Hospital mortality rate: {admissions['hospital_expire_flag'].mean()*100:.2f}%")

# Display sample
print(f"\nSample admission data:")
admissions.head(2)


STEP 1: Loading Core Patient and Admission Data

✓ Patients loaded: 100 records, 100 unique patients
✓ Admissions loaded: 275 records, 275 unique admissions

Admission Statistics:
- Average LOS: 6.88 days
- Median LOS: 4.85 days
- Hospital mortality rate: 5.45%

Sample admission data:


,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,los_days
0,10004235,24181354,2196-02-24 14:38:00,2196-03-04 14:02:00,NaT,URGENT,P03YMR,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicaid,ENGLISH,SINGLE,BLACK/CAPE VERDEAN,2196-02-24 12:15:00,2196-02-24 17:07:00,0,8.975000
1,10009628,25926192,2153-09-17 17:08:00,2153-09-25 13:20:00,NaT,URGENT,P41R5N,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicaid,?,MARRIED,HISPANIC/LATINO - PUERTO RICAN,NaT,NaT,0,7.841667


In [19]:
# ==============================================================================
# 2. LOAD AND PROCESS DIAGNOSES DATA
# ==============================================================================

print("\n" + "="*80)
print("STEP 2: Loading and Processing Diagnoses Data")
print("="*80)

# Load diagnoses
diagnoses_icd = pd.read_csv(HOSP_PATH / "diagnoses_icd.csv.gz")
d_icd_diagnoses = pd.read_csv(HOSP_PATH / "d_icd_diagnoses.csv.gz")

print(f"\n✓ Diagnoses loaded: {len(diagnoses_icd):,} records")
print(f"✓ ICD diagnosis dictionary loaded: {len(d_icd_diagnoses):,} unique codes")

# Merge diagnoses with descriptions
diagnoses_merged = diagnoses_icd.merge(
    d_icd_diagnoses,
    on=['icd_code', 'icd_version'],
    how='left'
)

# Count diagnoses per admission
diagnoses_per_admission = diagnoses_merged.groupby('hadm_id').size().reset_index(name='num_diagnoses')

print(f"\nDiagnosis Statistics:")
print(f"- Total unique ICD codes: {diagnoses_merged['icd_code'].nunique()}")
print(f"- ICD-9 codes: {(diagnoses_merged['icd_version']==9).sum():,}")
print(f"- ICD-10 codes: {(diagnoses_merged['icd_version']==10).sum():,}")
print(f"- Avg diagnoses per admission: {diagnoses_per_admission['num_diagnoses'].mean():.2f}")

# Top 10 most common diagnoses
print("\nTop 10 Most Common Diagnoses:")
top_diagnoses = diagnoses_merged.groupby(['icd_code', 'long_title']).size().reset_index(name='count').sort_values('count', ascending=False).head(10)
top_diagnoses



STEP 2: Loading and Processing Diagnoses Data

✓ Diagnoses loaded: 4,506 records
✓ ICD diagnosis dictionary loaded: 109,775 unique codes

Diagnosis Statistics:
- Total unique ICD codes: 1472
- ICD-9 codes: 2,193
- ICD-10 codes: 2,313
- Avg diagnoses per admission: 16.39

Top 10 Most Common Diagnoses:


,icd_code,long_title,count
208,4019,Unspecified essential hypertension,68
705,E785,"Hyperlipidemia, unspecified",57
93,2724,Other and unspecified hyperlipidemia,55
670,E039,"Hypothyroidism, unspecified",47
1414,Z794,Long term (current) use of insulin,37
1443,Z87891,Personal history of nicotine dependence,35
238,42731,Atrial fibrillation,34
75,25000,Diabetes mellitus without mention of complicat...,33
852,I2510,Atherosclerotic heart disease of native corona...,33
797,F329,"Major depressive disorder, single episode, uns...",32


In [20]:
# ==============================================================================
# 3. LOAD AND PROCESS PROCEDURES DATA
# ==============================================================================

print("\n" + "="*80)
print("STEP 3: Loading and Processing Procedures Data")
print("="*80)

# Load procedures
procedures_icd = pd.read_csv(HOSP_PATH / "procedures_icd.csv.gz")
d_icd_procedures = pd.read_csv(HOSP_PATH / "d_icd_procedures.csv.gz")

print(f"\n✓ Procedures loaded: {len(procedures_icd):,} records")
print(f"✓ ICD procedure dictionary loaded: {len(d_icd_procedures):,} unique codes")

# Merge procedures with descriptions
procedures_merged = procedures_icd.merge(
    d_icd_procedures,
    on=['icd_code', 'icd_version'],
    how='left'
)

# Convert chartdate
procedures_merged['chartdate'] = pd.to_datetime(procedures_merged['chartdate'])

# Count procedures per admission
procedures_per_admission = procedures_merged.groupby('hadm_id').size().reset_index(name='num_procedures')

print(f"\nProcedure Statistics:")
print(f"- Total unique procedure codes: {procedures_merged['icd_code'].nunique()}")
print(f"- Avg procedures per admission: {procedures_per_admission['num_procedures'].mean():.2f}")

# Top 10 most common procedures
print("\nTop 10 Most Common Procedures:")
top_procedures = procedures_merged.groupby(['icd_code', 'long_title']).size().reset_index(name='count').sort_values('count', ascending=False).head(10)
top_procedures



STEP 3: Loading and Processing Procedures Data

✓ Procedures loaded: 722 records
✓ ICD procedure dictionary loaded: 85,257 unique codes

Procedure Statistics:
- Total unique procedure codes: 352
- Avg procedures per admission: 3.86

Top 10 Most Common Procedures:


,icd_code,long_title,count
28,02HV33Z,Insertion of Infusion Device into Superior Ven...,23
190,3897,Central venous catheter placement with guidance,22
325,966,Enteral infusion of concentrated nutritional s...,18
326,9671,Continuous invasive mechanical ventilation for...,15
196,3961,Extracorporeal circulation auxiliary to open h...,13
189,3893,"Venous catheterization, not elsewhere classified",13
266,5491,Percutaneous abdominal drainage,12
322,9604,Insertion of endotracheal tube,11
210,3E0G76Z,Introduction of Nutritional Substance into Upp...,10
90,0BH17EZ,"Insertion of Endotracheal Airway into Trachea,...",10


In [21]:
# ==============================================================================
# 4. LOAD AND PROCESS PRESCRIPTION DATA
# ==============================================================================

print("\n" + "="*80)
print("STEP 4: Loading and Processing Prescription Data")
print("="*80)

# Load prescriptions
prescriptions = pd.read_csv(HOSP_PATH / "prescriptions.csv.gz")

print(f"\n✓ Prescriptions loaded: {len(prescriptions):,} records")

# Convert datetime columns
prescriptions['starttime'] = pd.to_datetime(prescriptions['starttime'])
prescriptions['stoptime'] = pd.to_datetime(prescriptions['stoptime'])

# Calculate prescription duration
prescriptions['duration_hours'] = (prescriptions['stoptime'] - prescriptions['starttime']).dt.total_seconds() / 3600

print(f"\nPrescription Statistics:")
print(f"- Total unique drugs: {prescriptions['drug'].nunique()}")
print(f"- Total unique admissions with prescriptions: {prescriptions['hadm_id'].nunique()}")
print(f"- Avg prescriptions per admission: {prescriptions.groupby('hadm_id').size().mean():.2f}")

# Count prescriptions per admission
prescriptions_per_admission = prescriptions.groupby('hadm_id').size().reset_index(name='num_prescriptions')

# Top 20 most prescribed drugs
print("\nTop 20 Most Prescribed Drugs:")
top_drugs = prescriptions['drug'].value_counts().head(20).reset_index()
top_drugs.columns = ['drug', 'count']
top_drugs



STEP 4: Loading and Processing Prescription Data

✓ Prescriptions loaded: 18,087 records

Prescription Statistics:
- Total unique drugs: 631
- Total unique admissions with prescriptions: 250
- Avg prescriptions per admission: 72.35

Top 20 Most Prescribed Drugs:


,drug,count
0,Insulin,915
1,0.9% Sodium Chloride,810
2,Potassium Chloride,610
3,Sodium Chloride 0.9% Flush,585
4,Furosemide,510
5,5% Dextrose,492
6,Bag,454
7,Magnesium Sulfate,402
8,Metoprolol Tartrate,371
9,Acetaminophen,344


In [22]:
# ==============================================================================
# 5. LOAD AND PROCESS LAB EVENTS DATA
# ==============================================================================

print("\n" + "="*80)
print("STEP 5: Loading and Processing Lab Events Data")
print("="*80)

# Load lab events (note: this can be a large file)
labevents = pd.read_csv(HOSP_PATH / "labevents.csv.gz")
d_labitems = pd.read_csv(HOSP_PATH / "d_labitems.csv.gz")

print(f"\n✓ Lab events loaded: {len(labevents):,} records")
print(f"✓ Lab item dictionary loaded: {len(d_labitems):,} unique lab tests")

# Merge with lab item descriptions
labevents_merged = labevents.merge(
    d_labitems,
    on='itemid',
    how='left'
)

# Convert datetime
labevents_merged['charttime'] = pd.to_datetime(labevents_merged['charttime'])

# Count lab tests per admission
labs_per_admission = labevents_merged.groupby('hadm_id').size().reset_index(name='num_lab_tests')

print(f"\nLab Events Statistics:")
print(f"- Total unique lab test types: {labevents_merged['itemid'].nunique()}")
print(f"- Total unique admissions with labs: {labevents_merged['hadm_id'].nunique()}")
print(f"- Avg lab tests per admission: {labs_per_admission['num_lab_tests'].mean():.2f}")

# Most common lab tests
print("\nTop 20 Most Common Lab Tests:")
top_labs = labevents_merged.groupby(['itemid', 'label']).size().reset_index(name='count').sort_values('count', ascending=False).head(20)
top_labs



STEP 5: Loading and Processing Lab Events Data

✓ Lab events loaded: 107,727 records
✓ Lab item dictionary loaded: 1,622 unique lab tests

Lab Events Statistics:
- Total unique lab test types: 498
- Total unique admissions with labs: 252
- Avg lab tests per admission: 314.71

Top 20 Most Common Lab Tests:


,itemid,label,count
121,50971,Potassium,3022
129,50983,Sodium,3007
81,50912,Creatinine,3003
71,50902,Chloride,2981
144,51006,Urea Nitrogen,2974
263,51221,Hematocrit,2913
57,50882,Bicarbonate,2863
45,50868,Anion Gap,2860
296,51265,Platelet Count,2827
264,51222,Hemoglobin,2787


In [23]:
# ==============================================================================
# 6. LOAD ICU STAYS DATA
# ==============================================================================

print("\n" + "="*80)
print("STEP 6: Loading ICU Stays Data")
print("="*80)

# Load ICU stays
icustays = pd.read_csv(ICU_PATH / "icustays.csv.gz")

print(f"\n✓ ICU stays loaded: {len(icustays):,} records")

# Convert datetime columns
icustays['intime'] = pd.to_datetime(icustays['intime'])
icustays['outtime'] = pd.to_datetime(icustays['outtime'])

print(f"\nICU Stay Statistics:")
print(f"- Total unique ICU stays: {icustays['stay_id'].nunique()}")
print(f"- Total unique patients with ICU stays: {icustays['subject_id'].nunique()}")
print(f"- Average ICU LOS: {icustays['los'].mean():.2f} days")
print(f"- Median ICU LOS: {icustays['los'].median():.2f} days")

# Care unit distribution
print("\nICU Care Unit Distribution:")
careunit_dist = icustays['first_careunit'].value_counts().reset_index()
careunit_dist.columns = ['care_unit', 'count']
careunit_dist



STEP 6: Loading ICU Stays Data

✓ ICU stays loaded: 140 records

ICU Stay Statistics:
- Total unique ICU stays: 140
- Total unique patients with ICU stays: 100
- Average ICU LOS: 3.68 days
- Median ICU LOS: 2.16 days

ICU Care Unit Distribution:


,care_unit,count
0,Surgical Intensive Care Unit (SICU),29
1,Medical Intensive Care Unit (MICU),29
2,Cardiac Vascular Intensive Care Unit (CVICU),25
3,Medical/Surgical Intensive Care Unit (MICU/SICU),23
4,Trauma SICU (TSICU),16
5,Coronary Care Unit (CCU),13
6,Neuro Surgical Intensive Care Unit (Neuro SICU),3
7,Neuro Stepdown,1
8,Neuro Intermediate,1


In [24]:
# ==============================================================================
# 7. LOAD ICU CHART EVENTS (VITAL SIGNS AND MEASUREMENTS)
# ==============================================================================

print("\n" + "="*80)
print("STEP 7: Loading ICU Chart Events Data")
print("="*80)

# Load chart events and items dictionary
chartevents = pd.read_csv(ICU_PATH / "chartevents.csv.gz")
d_items = pd.read_csv(ICU_PATH / "d_items.csv.gz")

print(f"\n✓ Chart events loaded: {len(chartevents):,} records")
print(f"✓ Item dictionary loaded: {len(d_items):,} unique items")

# Merge with item descriptions
chartevents_merged = chartevents.merge(
    d_items,
    on='itemid',
    how='left'
)

# Convert datetime
chartevents_merged['charttime'] = pd.to_datetime(chartevents_merged['charttime'])

print(f"\nChart Events Statistics:")
print(f"- Total unique item types: {chartevents_merged['itemid'].nunique()}")
print(f"- Total unique ICU stays with chart events: {chartevents_merged['stay_id'].nunique()}")

# Most common chart items
print("\nTop 20 Most Common Chart Items:")
top_chart_items = chartevents_merged.groupby(['itemid', 'label']).size().reset_index(name='count').sort_values('count', ascending=False).head(20)
top_chart_items



STEP 7: Loading ICU Chart Events Data

✓ Chart events loaded: 668,862 records
✓ Item dictionary loaded: 4,014 unique items

Chart Events Statistics:
- Total unique item types: 1318
- Total unique ICU stays with chart events: 140

Top 20 Most Common Chart Items:


,itemid,label,count
917,227969,Safety Measures,19330
1,220045,Heart Rate,13913
23,220210,Respiratory Rate,13913
30,220277,O2 saturation pulseoxymetry,13540
4,220048,Heart Rhythm,12460
346,224650,Ectopy Type 1,11044
20,220180,Non Invasive Blood Pressure diastolic,8349
19,220179,Non Invasive Blood Pressure systolic,8347
21,220181,Non Invasive Blood Pressure mean,8342
912,227958,Less Restrictive Measures,7675


In [25]:
# ==============================================================================
# 8. LOAD ICU INPUT/OUTPUT EVENTS
# ==============================================================================

print("\n" + "="*80)
print("STEP 8: Loading ICU Input/Output Events")
print("="*80)

# Load input events (medications, fluids administered)
inputevents = pd.read_csv(ICU_PATH / "inputevents.csv.gz")
print(f"\n✓ Input events loaded: {len(inputevents):,} records")

# Load output events (urine output, drains, etc.)
outputevents = pd.read_csv(ICU_PATH / "outputevents.csv.gz")
print(f"✓ Output events loaded: {len(outputevents):,} records")

# Convert datetime columns for inputevents
inputevents['starttime'] = pd.to_datetime(inputevents['starttime'])
inputevents['endtime'] = pd.to_datetime(inputevents['endtime'])

# Convert datetime columns for outputevents
outputevents['charttime'] = pd.to_datetime(outputevents['charttime'])

print(f"\nInput Events Statistics:")
print(f"- Total unique ICU stays with inputs: {inputevents['stay_id'].nunique()}")
print(f"- Total unique input item types: {inputevents['itemid'].nunique()}")

print(f"\nOutput Events Statistics:")
print(f"- Total unique ICU stays with outputs: {outputevents['stay_id'].nunique()}")
print(f"- Total unique output item types: {outputevents['itemid'].nunique()}")

# Most common input items (medications/fluids)
print("\nTop 20 Input Items:")
top_inputs = inputevents.merge(d_items, on='itemid', how='left')
top_input_summary = top_inputs.groupby(['itemid', 'label']).size().reset_index(name='count').sort_values('count', ascending=False).head(20)
top_input_summary



STEP 8: Loading ICU Input/Output Events

✓ Input events loaded: 20,404 records
✓ Output events loaded: 9,362 records

Input Events Statistics:
- Total unique ICU stays with inputs: 138
- Total unique input item types: 159

Output Events Statistics:
- Total unique ICU stays with outputs: 137
- Total unique output item types: 39

Top 20 Input Items:


,itemid,label,count
40,225158,NaCl 0.9%,2870
2,220949,Dextrose 5%,2358
94,225943,Solution,1287
24,221906,Norepinephrine,947
115,226452,PO Intake,805
29,222168,Propofol,794
48,225799,Gastric Meds,668
19,221749,Phenylephrine,625
116,226453,GT Flush,581
18,221744,Fentanyl,575


In [26]:
# ==============================================================================
# 9. CREATE COMPREHENSIVE PATIENT-LEVEL SUMMARY
# ==============================================================================

print("\n" + "="*80)
print("STEP 9: Creating Comprehensive Patient-Level Summary")
print("="*80)

# Start with patients
patient_summary = patients.copy()

# Merge with admission statistics
admission_stats = admissions.groupby('subject_id').agg({
    'hadm_id': 'count',
    'los_days': 'mean',
    'hospital_expire_flag': 'max'  # 1 if patient died in any admission
}).reset_index()
admission_stats.columns = ['subject_id', 'num_admissions', 'avg_los_days', 'ever_died_in_hospital']

patient_summary = patient_summary.merge(admission_stats, on='subject_id', how='left')

# Add diagnosis counts
diagnosis_counts = diagnoses_merged.groupby('subject_id').size().reset_index(name='total_diagnoses')
patient_summary = patient_summary.merge(diagnosis_counts, on='subject_id', how='left')

# Add procedure counts
procedure_counts = procedures_merged.groupby('subject_id').size().reset_index(name='total_procedures')
patient_summary = patient_summary.merge(procedure_counts, on='subject_id', how='left')

# Add prescription counts
prescription_counts = prescriptions.groupby('subject_id').size().reset_index(name='total_prescriptions')
patient_summary = patient_summary.merge(prescription_counts, on='subject_id', how='left')

# Add lab test counts
lab_counts = labevents_merged.groupby('subject_id').size().reset_index(name='total_lab_tests')
patient_summary = patient_summary.merge(lab_counts, on='subject_id', how='left')

# Add ICU stay information
icu_stats = icustays.groupby('subject_id').agg({
    'stay_id': 'count',
    'los': 'sum'
}).reset_index()
icu_stats.columns = ['subject_id', 'num_icu_stays', 'total_icu_los_days']
patient_summary = patient_summary.merge(icu_stats, on='subject_id', how='left')

# Fill NaN values with 0 for counts
count_columns = ['num_admissions', 'total_diagnoses', 'total_procedures', 'total_prescriptions', 
                 'total_lab_tests', 'num_icu_stays', 'total_icu_los_days']
patient_summary[count_columns] = patient_summary[count_columns].fillna(0)

# Fill NaN in binary columns
patient_summary['ever_died_in_hospital'] = patient_summary['ever_died_in_hospital'].fillna(0)

print(f"\n✓ Patient summary created with {len(patient_summary)} patients")
print(f"\nSummary Statistics:")
print(patient_summary.describe())

patient_summary.head(10)



STEP 9: Creating Comprehensive Patient-Level Summary

✓ Patient summary created with 100 patients

Summary Statistics:
         subject_id  anchor_age  anchor_year  num_admissions  avg_los_days  \
count  1.000000e+02   100.00000   100.000000        100.0000    100.000000   
mean   1.001878e+07    61.75000  2148.680000          2.7500      7.205877   
std    1.115115e+04    16.16979    25.629008          3.1794      4.692535   
min    1.000003e+07    21.00000  2110.000000          1.0000      1.444444   
25%    1.000889e+07    51.75000  2129.000000          1.0000      4.253950   
50%    1.001909e+07    63.00000  2145.500000          1.0000      5.847569   
75%    1.002550e+07    72.00000  2172.500000          3.0000      9.031337   
max    1.004002e+07    91.00000  2201.000000         20.0000     31.307639   

       ever_died_in_hospital  total_diagnoses  total_procedures  \
count              100.00000       100.000000        100.000000   
mean                 0.15000        45.0600

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,num_admissions,avg_los_days,ever_died_in_hospital,total_diagnoses,total_procedures,total_prescriptions,total_lab_tests,num_icu_stays,total_icu_los_days
0,10014729,F,21,2125,2011 - 2013,NaN,2,8.079514,0,20,10.0,257,621,1,2.471169
1,10003400,F,72,2134,2011 - 2013,2137-09-02,7,11.737004,1,153,25.0,675,3613,3,31.897847
2,10002428,F,80,2155,2011 - 2013,NaN,7,5.621230,0,114,17.0,320,2177,4,25.015093
3,10032725,F,38,2143,2011 - 2013,2143-03-30,2,15.227778,0,37,7.0,258,889,1,3.349687
4,10027445,F,48,2142,2011 - 2013,2146-02-09,3,9.781250,0,55,9.0,230,973,2,5.016412
5,10037928,F,78,2175,2011 - 2013,NaN,10,4.142083,0,169,4.0,429,2372,2,1.653773
6,10001725,F,46,2110,2011 - 2013,NaN,1,2.994444,0,18,3.0,70,572,1,1.338588
7,10040025,F,64,2143,2011 - 2013,2148-02-07,10,6.586736,0,259,16.0,715,2561,1,6.538102
8,10008454,F,26,2110,2011 - 2013,NaN,1,10.390278,0,11,1.0,139,348,1,4.983889
9,10020640,F,91,2153,2011 - 2013,2154-02-04,1,7.562500,0,21,0.0,88,369,1,2.557604


In [27]:
# ==============================================================================
# 10. CREATE ADMISSION-LEVEL SUMMARY
# ==============================================================================

print("\n" + "="*80)
print("STEP 10: Creating Comprehensive Admission-Level Summary")
print("="*80)

# Start with admissions
admission_summary = admissions.copy()

# Add patient demographics
admission_summary = admission_summary.merge(
    patients[['subject_id', 'gender', 'anchor_age']],
    on='subject_id',
    how='left'
)

# Add diagnosis counts per admission
diag_counts = diagnoses_per_admission
admission_summary = admission_summary.merge(diag_counts, on='hadm_id', how='left')

# Add procedure counts per admission
proc_counts = procedures_per_admission
admission_summary = admission_summary.merge(proc_counts, on='hadm_id', how='left')

# Add prescription counts per admission
presc_counts = prescriptions_per_admission
admission_summary = admission_summary.merge(presc_counts, on='hadm_id', how='left')

# Add lab test counts per admission
lab_counts_adm = labs_per_admission
admission_summary = admission_summary.merge(lab_counts_adm, on='hadm_id', how='left')

# Add ICU stay information per admission
icu_per_admission = icustays.groupby('hadm_id').agg({
    'stay_id': 'count',
    'los': 'sum'
}).reset_index()
icu_per_admission.columns = ['hadm_id', 'num_icu_stays', 'total_icu_los']
admission_summary = admission_summary.merge(icu_per_admission, on='hadm_id', how='left')

# Fill NaN values
count_cols = ['num_diagnoses', 'num_procedures', 'num_prescriptions', 'num_lab_tests', 'num_icu_stays', 'total_icu_los']
admission_summary[count_cols] = admission_summary[count_cols].fillna(0)

print(f"\n✓ Admission summary created with {len(admission_summary)} admissions")
print(f"\nAdmission-Level Summary Statistics:")
print(admission_summary[['los_days', 'num_diagnoses', 'num_procedures', 'num_prescriptions', 'num_lab_tests']].describe())

admission_summary.head(10)



STEP 10: Creating Comprehensive Admission-Level Summary

✓ Admission summary created with 275 admissions

Admission-Level Summary Statistics:
         los_days  num_diagnoses  num_procedures  num_prescriptions  \
count  275.000000     275.000000      275.000000         275.000000   
mean     6.875530      16.385455        2.625455          65.770909   
std      6.733832       9.014413        3.559926          60.946505   
min      0.046528       2.000000        0.000000           0.000000   
25%      2.692014       9.000000        0.000000          25.000000   
50%      4.852778      14.000000        1.000000          48.000000   
75%      8.768403      23.000000        4.000000          92.000000   
max     44.927778      39.000000       23.000000         348.000000   

       num_lab_tests  
count     275.000000  
mean      288.389091  
std       390.057021  
min         0.000000  
25%        63.000000  
50%       163.000000  
75%       350.500000  
max      2538.000000  


,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,los_days,gender,anchor_age,num_diagnoses,num_procedures,num_prescriptions,num_lab_tests,num_icu_stays,total_icu_los
0,10004235,24181354,2196-02-24 14:38:00,2196-03-04 14:02:00,NaT,URGENT,P03YMR,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicaid,ENGLISH,SINGLE,BLACK/CAPE VERDEAN,2196-02-24 12:15:00,2196-02-24 17:07:00,0,8.975000,M,47,26,5.0,195.0,843.0,1.0,4.952106
1,10009628,25926192,2153-09-17 17:08:00,2153-09-25 13:20:00,NaT,URGENT,P41R5N,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicaid,?,MARRIED,HISPANIC/LATINO - PUERTO RICAN,NaT,NaT,0,7.841667,M,58,8,3.0,166.0,387.0,1.0,2.280752
2,10018081,23983182,2134-08-18 02:02:00,2134-08-23 19:35:00,NaT,URGENT,P233F6,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE,2134-08-17 16:24:00,2134-08-18 03:15:00,0,5.731250,M,79,19,2.0,59.0,259.0,0.0,0.000000
3,10006053,22942076,2111-11-13 23:39:00,2111-11-15 17:20:00,2111-11-15 17:20:00,URGENT,P38TI6,TRANSFER FROM HOSPITAL,DIED,Medicaid,ENGLISH,NaN,UNKNOWN,NaT,NaT,1,1.736806,M,52,10,3.0,63.0,313.0,2.0,1.775093
4,10031404,21606243,2113-08-04 18:46:00,2113-08-06 20:57:00,NaT,URGENT,P07HDB,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,NaT,NaT,0,2.090972,F,82,15,1.0,34.0,57.0,1.0,1.206481
5,10005817,20626031,2132-12-12 01:43:00,2132-12-20 15:04:00,NaT,URGENT,P41R5N,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicare,ENGLISH,MARRIED,WHITE,NaT,NaT,0,8.556250,M,66,26,3.0,194.0,409.0,1.0,2.359097
6,10019385,20297618,2180-02-15 20:28:00,2180-02-25 13:45:00,NaT,URGENT,P536JC,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,NaT,NaT,0,9.720139,M,44,9,2.0,114.0,459.0,1.0,1.313287
7,10002495,24982426,2141-05-22 20:17:00,2141-05-29 17:41:00,NaT,URGENT,P79SJ2,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,UNKNOWN,NaT,NaT,0,6.891667,M,81,26,7.0,113.0,391.0,1.0,5.087512
8,10038081,20755971,2115-09-27 20:40:00,2115-10-12 00:00:00,2115-10-12 22:20:00,URGENT,P48GIG,TRANSFER FROM HOSPITAL,DIED,Other,?,SINGLE,UNKNOWN,NaT,NaT,1,14.138889,F,63,38,9.0,99.0,919.0,1.0,3.698519
9,10019917,22585261,2182-01-07 23:25:00,2182-01-10 16:52:00,NaT,URGENT,P3529J,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,SINGLE,OTHER,NaT,NaT,0,2.727083,M,44,4,2.0,18.0,75.0,1.0,1.321516


In [28]:
# ==============================================================================
# 11. EXTRACT KEY VITAL SIGNS FROM CHART EVENTS
# ==============================================================================

print("\n" + "="*80)
print("STEP 11: Extracting Key Vital Signs from Chart Events")
print("="*80)

# Define key vital sign itemids (common in MIMIC-IV)
vital_sign_items = {
    220045: 'Heart Rate',
    220050: 'Arterial Blood Pressure systolic',
    220051: 'Arterial Blood Pressure diastolic',
    220052: 'Arterial Blood Pressure mean',
    220179: 'Non Invasive Blood Pressure systolic',
    220180: 'Non Invasive Blood Pressure diastolic',
    220181: 'Non Invasive Blood Pressure mean',
    220210: 'Respiratory Rate',
    223761: 'Temperature Fahrenheit',
    223762: 'Temperature Celsius',
    220277: 'SpO2'
}

# Filter chartevents for vital signs
vital_signs = chartevents_merged[chartevents_merged['itemid'].isin(vital_sign_items.keys())].copy()

# Add readable vital sign names
vital_signs['vital_sign'] = vital_signs['itemid'].map(vital_sign_items)

print(f"\n✓ Extracted {len(vital_signs):,} vital sign measurements")
print(f"- Unique ICU stays with vital signs: {vital_signs['stay_id'].nunique()}")

# Calculate statistics for each vital sign
vital_stats = vital_signs.groupby('vital_sign').agg({
    'valuenum': ['count', 'mean', 'std', 'min', 'max']
}).round(2)

print(f"\nVital Sign Statistics:")
print(vital_stats)

# Create a pivot table with average vital signs per ICU stay
vital_pivot = vital_signs.groupby(['stay_id', 'vital_sign'])['valuenum'].agg(['mean', 'min', 'max']).reset_index()
vital_pivot.head(20)



STEP 11: Extracting Key Vital Signs from Chart Events

✓ Extracted 86,783 vital sign measurements
- Unique ICU stays with vital signs: 140

Vital Sign Statistics:
                                      valuenum                            
                                         count    mean    std   min    max
vital_sign                                                                
Arterial Blood Pressure diastolic         5524   59.47  10.92  21.0  165.0
Arterial Blood Pressure mean              5560   77.66  20.10 -23.0  801.0
Arterial Blood Pressure systolic          5525  113.82  20.05  25.0  207.0
Heart Rate                               13913   91.12  18.69   0.0  200.0
Non Invasive Blood Pressure diastolic     8349   63.34  14.93  18.0  162.0
Non Invasive Blood Pressure mean          8342   75.49  15.19  22.0  171.0
Non Invasive Blood Pressure systolic      8347  115.27  21.76  46.0  215.0
Respiratory Rate                         13913   19.95   5.55   0.0   58.0
SpO2       

,stay_id,vital_sign,mean,min,max
0,30057454,Arterial Blood Pressure diastolic,62.391892,45.0,78.0
1,30057454,Arterial Blood Pressure mean,72.310811,54.0,89.0
2,30057454,Arterial Blood Pressure systolic,94.371622,66.0,116.0
3,30057454,Heart Rate,111.423841,100.0,200.0
4,30057454,Non Invasive Blood Pressure diastolic,61.571429,47.0,78.0
5,30057454,Non Invasive Blood Pressure mean,67.357143,50.0,85.0
6,30057454,Non Invasive Blood Pressure systolic,88.500000,58.0,113.0
7,30057454,Respiratory Rate,15.649007,4.0,27.0
8,30057454,SpO2,93.701987,87.0,98.0
9,30057454,Temperature Fahrenheit,98.464000,97.7,99.7


In [30]:
# ==============================================================================
# 12. EXTRACT KEY LAB RESULTS
# ==============================================================================

print("\n" + "="*80)
print("STEP 12: Extracting Key Lab Results")
print("="*80)

# Define key lab test itemids (common in MIMIC-IV)
key_lab_items = {
    50868: 'Anion Gap',
    50882: 'Bicarbonate',
    50893: 'Calcium Total',
    50902: 'Chloride',
    50912: 'Creatinine',
    50931: 'Glucose',
    50960: 'Magnesium',
    50970: 'Phosphate',
    50971: 'Potassium',
    50983: 'Sodium',
    51006: 'Urea Nitrogen',
    51221: 'Hematocrit',
    51222: 'Hemoglobin',
    51248: 'MCH',
    51249: 'MCHC',
    51250: 'MCV',
    51265: 'Platelet Count',
    51277: 'RDW',
    51279: 'Red Blood Cells',
    51301: 'White Blood Cells'
}

# Filter labevents for key labs
key_labs = labevents_merged[labevents_merged['itemid'].isin(key_lab_items.keys())].copy()

# Add readable lab names
key_labs['lab_name'] = key_labs['itemid'].map(key_lab_items)

print(f"\n✓ Extracted {len(key_labs):,} key lab test results")
print(f"- Unique admissions with key labs: {key_labs['hadm_id'].nunique()}")

# Calculate statistics for each lab test
lab_stats = key_labs.groupby('lab_name').agg({
    'valuenum': ['count', 'mean', 'std', 'min', 'max']
}).round(2)

print(f"\nKey Lab Test Statistics:")
print(lab_stats)

# Create first and last lab values per admission
first_labs = key_labs.sort_values('charttime').groupby(['hadm_id', 'lab_name']).first()['valuenum'].reset_index()
first_labs.columns = ['hadm_id', 'lab_name', 'first_value']

last_labs = key_labs.sort_values('charttime').groupby(['hadm_id', 'lab_name']).last()['valuenum'].reset_index()
last_labs.columns = ['hadm_id', 'lab_name', 'last_value']

# Combine first and last
lab_summary = first_labs.merge(last_labs, on=['hadm_id', 'lab_name'], how='outer')

print(f"\n✓ Lab summary created with first and last values per admission")
lab_summary.head(20)



STEP 12: Extracting Key Lab Results

✓ Extracted 55,692 key lab test results
- Unique admissions with key labs: 251

Key Lab Test Statistics:
                  valuenum                                
                     count    mean     std     min     max
lab_name                                                  
Anion Gap             2860   14.95    4.30    4.00   40.00
Bicarbonate           2863   24.88    4.89    5.00   46.00
Calcium Total         2377    8.57    0.79    2.70   13.00
Chloride              2981  101.07    6.45   80.00  136.00
Creatinine            3003    1.54    1.48    0.10   15.20
Glucose               2711  144.03   75.96   29.00  996.00
Hematocrit            2908   30.78    6.59   10.10   55.20
Hemoglobin            2785   10.06    2.22    3.30   18.70
MCH                   2748   29.86    2.78   19.40   37.90
MCHC                  2748   32.60    1.63   26.20   37.90
MCV                   2748   91.67    7.80   68.00  124.00
Magnesium             2470    2

,hadm_id,lab_name,first_value,last_value
0,20044587.0,Anion Gap,9.0,10.00
1,20044587.0,Bicarbonate,23.0,30.00
2,20044587.0,Calcium Total,8.6,8.60
3,20044587.0,Chloride,117.0,103.00
4,20044587.0,Creatinine,1.0,0.80
5,20044587.0,Glucose,76.0,107.00
6,20044587.0,Hematocrit,31.4,26.10
7,20044587.0,Hemoglobin,10.8,8.80
8,20044587.0,MCH,31.7,32.00
9,20044587.0,MCHC,34.2,33.70


In [31]:
# ==============================================================================
# 13. SAVE PREPROCESSED DATA
# ==============================================================================

print("\n" + "="*80)
print("STEP 13: Saving Preprocessed Data")
print("="*80)

# Save patient-level summary
patient_summary.to_csv(OUTPUT_PATH / "mimic_patient_summary.csv", index=False)
print(f"\n✓ Saved: mimic_patient_summary.csv ({len(patient_summary)} records)")

# Save admission-level summary
admission_summary.to_csv(OUTPUT_PATH / "mimic_admission_summary.csv", index=False)
print(f"✓ Saved: mimic_admission_summary.csv ({len(admission_summary)} records)")

# Save diagnoses with descriptions
diagnoses_merged.to_csv(OUTPUT_PATH / "mimic_diagnoses_detailed.csv", index=False)
print(f"✓ Saved: mimic_diagnoses_detailed.csv ({len(diagnoses_merged)} records)")

# Save procedures with descriptions
procedures_merged.to_csv(OUTPUT_PATH / "mimic_procedures_detailed.csv", index=False)
print(f"✓ Saved: mimic_procedures_detailed.csv ({len(procedures_merged)} records)")

# Save prescriptions
prescriptions.to_csv(OUTPUT_PATH / "mimic_prescriptions.csv", index=False)
print(f"✓ Saved: mimic_prescriptions.csv ({len(prescriptions)} records)")

# Save key lab results
key_labs.to_csv(OUTPUT_PATH / "mimic_key_labs.csv", index=False)
print(f"✓ Saved: mimic_key_labs.csv ({len(key_labs)} records)")

# Save lab summary (first/last per admission)
lab_summary.to_csv(OUTPUT_PATH / "mimic_lab_summary.csv", index=False)
print(f"✓ Saved: mimic_lab_summary.csv ({len(lab_summary)} records)")

# Save ICU stays
icustays.to_csv(OUTPUT_PATH / "mimic_icustays.csv", index=False)
print(f"✓ Saved: mimic_icustays.csv ({len(icustays)} records)")

# Save vital signs
vital_signs.to_csv(OUTPUT_PATH / "mimic_vital_signs.csv", index=False)
print(f"✓ Saved: mimic_vital_signs.csv ({len(vital_signs)} records)")

# Save vital signs pivot (summary)
vital_pivot.to_csv(OUTPUT_PATH / "mimic_vital_signs_summary.csv", index=False)
print(f"✓ Saved: mimic_vital_signs_summary.csv ({len(vital_pivot)} records)")

print("\n" + "="*80)
print("PREPROCESSING COMPLETE!")
print("="*80)
print(f"\nAll preprocessed files saved to: {OUTPUT_PATH}")



STEP 13: Saving Preprocessed Data

✓ Saved: mimic_patient_summary.csv (100 records)
✓ Saved: mimic_admission_summary.csv (275 records)
✓ Saved: mimic_diagnoses_detailed.csv (4506 records)
✓ Saved: mimic_procedures_detailed.csv (722 records)
✓ Saved: mimic_prescriptions.csv (18087 records)
✓ Saved: mimic_key_labs.csv (55692 records)
✓ Saved: mimic_lab_summary.csv (4912 records)
✓ Saved: mimic_icustays.csv (140 records)
✓ Saved: mimic_vital_signs.csv (86783 records)
✓ Saved: mimic_vital_signs_summary.csv (1184 records)

PREPROCESSING COMPLETE!

All preprocessed files saved to: output


# MIMIC-IV Data Visualization

This section provides visualizations of the preprocessed MIMIC-IV data.


# Summary Report

## Dataset Structure

The MIMIC-IV demo dataset contains **100 patients** with the following structure:

### Hospital Module (`hosp/`)
- **patients.csv.gz**: Patient demographics (age, gender)
- **admissions.csv.gz**: Hospital admission records
- **diagnoses_icd.csv.gz**: ICD diagnosis codes
- **procedures_icd.csv.gz**: ICD procedure codes
- **prescriptions.csv.gz**: Medication prescriptions
- **labevents.csv.gz**: Laboratory test results
- **microbiologyevents.csv.gz**: Microbiology culture results
- **emar.csv.gz**: Electronic medication administration records
- **pharmacy.csv.gz**: Pharmacy orders
- **poe.csv.gz**: Provider order entries
- **transfers.csv.gz**: Patient transfer records
- **services.csv.gz**: Clinical service records
- **drgcodes.csv.gz**: DRG billing codes

### ICU Module (`icu/`)
- **icustays.csv.gz**: ICU stay records
- **chartevents.csv.gz**: Vital signs and measurements
- **inputevents.csv.gz**: Medications and fluids administered
- **outputevents.csv.gz**: Output measurements (urine, drains)
- **procedureevents.csv.gz**: ICU procedures
- **ingredientevents.csv.gz**: Medication ingredients
- **datetimeevents.csv.gz**: Date/time events

### Dictionary Files
- **d_icd_diagnoses.csv.gz**: ICD diagnosis code descriptions
- **d_icd_procedures.csv.gz**: ICD procedure code descriptions
- **d_labitems.csv.gz**: Lab test descriptions
- **d_items.csv.gz**: ICU item descriptions
- **d_hcpcs.csv.gz**: HCPCS code descriptions

## Key Preprocessing Steps

1. **Data Loading**: Loaded all core tables with proper compression handling
2. **Datetime Conversion**: Converted all timestamp columns to pandas datetime
3. **Data Merging**: Joined fact tables with dimension tables for descriptions
4. **Feature Engineering**: 
   - Calculated length of stay
   - Extracted first/last lab values
   - Aggregated vital signs
   - Counted events per patient/admission
5. **Summary Creation**:
   - Patient-level summary with all clinical metrics
   - Admission-level summary with detailed statistics
6. **Data Export**: Saved all preprocessed data to CSV files

## Output Files Generated

All files saved to `output/` directory:
- `mimic_patient_summary.csv`: Comprehensive patient-level data
- `mimic_admission_summary.csv`: Comprehensive admission-level data
- `mimic_diagnoses_detailed.csv`: Diagnoses with descriptions
- `mimic_procedures_detailed.csv`: Procedures with descriptions
- `mimic_prescriptions.csv`: Prescription data
- `mimic_key_labs.csv`: Key lab test results
- `mimic_lab_summary.csv`: First/last lab values per admission
- `mimic_icustays.csv`: ICU stay data
- `mimic_vital_signs.csv`: Vital sign measurements
- `mimic_vital_signs_summary.csv`: Vital sign statistics per stay
- `mimic_overview.png`: Overview visualizations
- `mimic_clinical_analysis.png`: Clinical data visualizations
